In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# SOM class

In [2]:
class SOM():
    def __init__(self, n, m, input_size, nbhd_fun_name='gaussian', nbhd_width=1.0, lr_decay_rate=0.1, grid_type='rect'):
        self.size = (n, m)
        self.input_size = input_size
        self.weights = np.random.rand(n, m, input_size)
        self.nbhd_width = nbhd_width
        self._set_nbhd_fun(nbhd_fun_name)
        self.lr_decay_rate = lr_decay_rate
        self.lr_fun = lambda t: np.exp(-t / self.lr_decay_rate)
        self.grid_type = grid_type

    def _set_nbhd_fun(self, nbhd_fun_name):
        """Set the neighborhood function based on the specified type."""
        if nbhd_fun_name == 'gaussian':
            self.nbhd_fun = lambda d: np.exp(- d ** 2 / (2 * self.nbhd_width))
        elif nbhd_fun_name == 'mexican_hat':
            self.nbhd_fun = lambda d: (1 - (d ** 2) / self.nbhd_width) * np.exp(- (d ** 2) / (2 * self.nbhd_width))
        else:
            raise ValueError("Unknown neighborhood function: {}".format(nbhd_fun_name))
    
    def find_bmu(self, x):
        """Find the Best Matching Unit (BMU) for the input x."""
        bmu_flat_idx = np.argmin(np.linalg.norm(self.weights - x, axis=2))
        bmu = np.unravel_index(bmu_flat_idx, self.size)
        if self.grid_type == 'hex':
            bmu = (bmu[0] + 0.5 * (bmu[1] % 2), bmu[1])
        return bmu
    
    def _update_weights(self, x, bmu, t):
        """Update the weights of the SOM."""
        learning_rate = self.lr_fun(t)
        for i in range(self.size[0]):
            for j in range(self.size[1]):
                i_hex = i
                if self.grid_type == 'hex':
                    if j % 2 == 1:
                        i_hex = i + 0.5
                d = np.linalg.norm(np.array(bmu) - np.array((i_hex, j)))
                nbhd_weight = self.nbhd_fun(d)
                self.weights[i, j] += learning_rate * nbhd_weight * (x - self.weights[i, j])
        
    def fit(self, data, epochs=1000, n_display_epochs=100):
        """Train the SOM with the given data."""
        data = np.array(data)
        for t in range(epochs):
            if (t+1) % n_display_epochs == 0 or t == 0:
                print(f"Epoch {t+1}/{epochs}")
            shuffled_data = np.random.permutation(data)
            for x in shuffled_data:
                bmu = self.find_bmu(x)
                self._update_weights(x, bmu, t)

    def u_matrix(self):
        """Calculate the distance map of the SOM."""
        rows, cols = self.size
        u_matrix = np.zeros((self.size[0], self.size[1]))

        neighbor_offsets = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        for i in range(rows):
            for j in range(cols):
                neighbor_distances = []
                for dx, dy in neighbor_offsets:
                    ni, nj = i + dx, j + dy
                    if 0 <= ni < rows and 0 <= nj < cols:
                        dist = np.linalg.norm(self.weights[i, j] - self.weights[ni, nj])
                        neighbor_distances.append(dist)
                u_matrix[i, j] = np.mean(neighbor_distances) if neighbor_distances else 0
        u_matrix = (u_matrix - np.min(u_matrix)) / (np.max(u_matrix) - np.min(u_matrix))

        return u_matrix

In [3]:
def find_clusters(som, X, y, max_clusters=20, n_labels=10):
    """Find clusters in the SOM using KMeans."""
    scores = []
    labels_list = []
    weights = som.weights.reshape(-1, som.input_size)

    for k in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        labels = kmeans.fit_predict(weights)
        score = silhouette_score(weights, labels)
        scores.append(score)
        labels_list.append(labels)

    best_k = np.argmax(scores) + 2
    neuron_labels = labels_list[best_k - 2]
    neuron_labels_2d = neuron_labels.reshape(som.size)

    def assign_cluster(x):
        bmu = som.find_bmu(x)
        if som.grid_type == 'hex':
            bmu = (int(bmu[0] - 0.5 * (bmu[1] % 2)), bmu[1])
        return neuron_labels_2d[bmu]
    
    clusters = np.array([assign_cluster(x) for x in X])

    if best_k == np.unique(y).size:
        cluster_to_label = {}
        for cluster in np.unique(clusters):
            labels_in_cluster = y[clusters == cluster]
            if len(labels_in_cluster) > 0:
                most_common_label = pd.Series(labels_in_cluster).mode()[0]
                cluster_to_label[cluster] = most_common_label
        y_pred = np.array([cluster_to_label[c] for c in clusters])
    else:
        print("Warning: The number of clusters does not match the number of labels. No label assignment will be done.")
        y_pred = clusters
    return y_pred, best_k, scores[best_k - 2], neuron_labels_2d


In [4]:
def normalize(data, min = None, max = None):
    data = np.array(data)
    if min is None:
        min = np.min(data, axis=0)

    if max is None:
        max = np.max(data, axis=0)

    return (data - min) / (max - min), min, max    

In [5]:
def denormalize(data, min, max):
    data = np.array(data)
    return data * (max - min) + min

# MNIST

In [6]:
from sklearn.datasets import fetch_openml

# Load the MNIST dataset
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data.values
y = mnist.target.astype(int)
X = X / 255.0  # Normalize the data

In [7]:
print("Data shape:", X.shape, "Labels shape:", y.shape)

Data shape: (70000, 784) Labels shape: (70000,)


In [8]:
print('Target classes:', np.unique(y))

Target classes: [0 1 2 3 4 5 6 7 8 9]


In [9]:
from sklearn.model_selection import train_test_split
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.99, random_state=42)

In [10]:
print("Data shape:", X.shape)
print("Train shape:", X_train.shape, "Labels shape:", y_train.shape)

Data shape: (70000, 784)
Train shape: (700, 784) Labels shape: (700,)


In [11]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=20)
# X_train = pca.fit_transform(X_train)

In [12]:
som_rect = SOM(20, 20, 784, nbhd_fun_name='gaussian', nbhd_width=1.0, lr_decay_rate=1000, grid_type='rect')
som_rect.fit(X_train, epochs=100, n_display_epochs=1)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [13]:
y_pred_rect, best_k_rect, score_rect, neuron_labels_2d_rect = find_clusters(som_rect, X_train, y_train, max_clusters=20, n_labels=10)

In [14]:
print(best_k_rect)

20


In [15]:
som_hex = SOM(20, 20, 784, nbhd_fun_name='gaussian', nbhd_width=1.0, lr_decay_rate=1000, grid_type='hex')
som_hex.fit(X_train, epochs=100, n_display_epochs=1)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [16]:
y_pred_hex, best_k_hex, score_hex, neuron_labels_2d_hex = find_clusters(som_hex, X_train, y_train, max_clusters=20, n_labels=10)

In [17]:
print(best_k_rect)

20


# HAR

In [18]:
X = pd.read_csv('data/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None).values
y = pd.read_csv('data/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None).values

C:\Users\kuba_\AppData\Local\Temp\ipykernel_30784\3976687524.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X = pd.read_csv('data/UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None).values
C:\Users\kuba_\AppData\Local\Temp\ipykernel_30784\3976687524.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv('data/UCI HAR Dataset/train/y_train.txt', delim_whitespace=True, header=None).values


In [19]:
print("Data shape:", X.shape, "Labels shape:", y.shape)

Data shape: (7352, 561) Labels shape: (7352, 1)


In [20]:
from sklearn.model_selection import train_test_split
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.7, random_state=42)

In [21]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=20)
# X_train = pca.fit_transform(X_train)

In [22]:
print("Data shape:", X_train.shape, "Labels shape:", y_train.shape)

Data shape: (2205, 561) Labels shape: (2205, 1)


In [24]:
print('Unique labels:', np.unique(y_train))

Unique labels: [1 2 3 4 5 6]


In [26]:
X_train, min, max = normalize(X_train)

In [27]:
som_rect = SOM(20, 20, 561, nbhd_fun_name='gaussian', nbhd_width=1.0, lr_decay_rate=1000, grid_type='rect')
som_rect.fit(X_train, epochs=100, n_display_epochs=1)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [28]:
y_pred_rect, best_k_rect, score_rect, neuron_labels_2d_rect = find_clusters(som_rect, X_train, y_train, max_clusters=20, n_labels=6)

In [29]:
print(best_k_rect)

2


In [30]:
som_hex = SOM(20, 20, 561, nbhd_fun_name='gaussian', nbhd_width=1.0, lr_decay_rate=1000, grid_type='hex')
som_hex.fit(X_train, epochs=100, n_display_epochs=1)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [31]:
y_pred_hex, best_k_hex, score_hex, neuron_labels_2d_hex = find_clusters(som_hex, X_train, y_train, max_clusters=20, n_labels=6)

In [32]:
print(best_k_hex)

2
